# CNN-AE
Convolutional AutoEncoder

In [1]:
#library import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime as dt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, Conv2D, Conv2DTranspose
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split

#definition function            
def MSE_loss(tr_value, model):
    prediction = model.predict(tr_value)
    loss = MeanSquaredError()
    return loss(tr_value, prediction)

def norm(df, mean=None, std=None, train=True):
    if train == True:
        mean = df.mean()
        std = df.std()
        
    norm_df = (df - mean)/std
    return norm_df, mean, std

def CsvToTensor(Dataframe1, mean_std_data=None, train=True):
    dataset = Dataframe1.values[2:].astype('float32')
    columns = Dataframe1.values[0]
    df = pd.DataFrame(data=dataset, columns=columns)
    
    if train == True:
        df1, mean1, std1 = norm(df[['roll','pitch','yaw']], train)
        df2, mean2, std2 = norm(df[['ax','ay','az']], train)
        df3, mean3, std3 = norm(df[['gx','gy','gz']], train)
        mean_std_data = (mean1, std1, mean2, std2, mean3, std3)
        
    else:
        mean1, std1, mean2, std2, mean3, std3 = mean_std_data
        df1, _, _ = norm(df[['roll','pitch','yaw']], mean1, std1, train)
        df2, _, _ = norm(df[['ax','ay','az']], mean2, std2, train)
        df3, _, _ = norm(df[['gx','gy','gz']], mean3, std3, train)
        
    df1 = df1.fillna(0)
    df2 = df2.fillna(0)
    df3 = df3.fillna(0)
    
    ch1 = []
    ch2 = []
    ch3 = []

    for i in range(len(df1)//5):
        ch1.append(df1[i:i+5])
    for i in range(len(df2)//5):
        ch2.append(df2[i:i+5])
    for i in range(len(df3)//5):
        ch3.append(df3[i:i+5])

    ch1 = np.array(ch1)
    ch2 = np.array(ch2)
    ch3 = np.array(ch3)

    img = np.array((ch1,ch2,ch3)).transpose(1,2,3,0)
    
    return img, mean_std_data

In [2]:
#define AE class
class Autoencoder(Model):
    """Convolutional Autoencoder Model"""
    def __init__(self, img_shape):
        super(Autoencoder, self).__init__()
        
        #encoder
        self.encoder = Sequential()
        self.encoder.add(Input(shape=img_shape[1:]))
        self.encoder.add(Conv2D(filters=10, kernel_size=3,activation='relu',padding='same'))
        self.encoder.add(Conv2D(filters=8, kernel_size=3,activation='relu',padding='same'))
        
        #decoder
        self.decoder = Sequential()
        self.decoder.add(Conv2DTranspose(8, kernel_size=3,activation='relu',padding='same'))
        self.decoder.add(Conv2DTranspose(10, kernel_size=3,activation='relu',padding='same'))
        self.decoder.add(Conv2D(3, kernel_size=3,activation='linear',padding='same'))
        
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [3]:
#main
steady_state = './cap_data/data9.csv'
accident_state = './cap_data/accident2.csv'

data1 = pd.read_csv(steady_state, sep=',', header=None)
tensor_data, tr_m_data = CsvToTensor(data1)
X_train, X_test = train_test_split(tensor_data, test_size = 0.2,shuffle=True)

AE_model = Autoencoder(X_train.shape)
AE_model.compile(optimizer='adam', loss=MeanSquaredError())
AE_model.fit(X_train,X_train,epochs=100,shuffle=True,validation_data=(X_test, X_test))
AE_model.summary()

X_train.shape

Epoch 1/100
123/123 [==============================] - 1s 5ms/step - loss: 0.3577 - val_loss: 0.1310
Epoch 2/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0985 - val_loss: 0.0718
Epoch 3/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0582 - val_loss: 0.0475
Epoch 4/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0406 - val_loss: 0.0363
Epoch 5/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0315 - val_loss: 0.0290
Epoch 6/100
123/123 [==============================] - 0s 3ms/step - loss: 0.0255 - val_loss: 0.0235
Epoch 7/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0222 - val_loss: 0.0227
Epoch 8/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0192 - val_loss: 0.0192
Epoch 9/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0177 - val_loss: 0.0166
Epoch 10/100
123/123 [==============================] - 0s 3ms/step - loss: 0.0159 - val_lo

Epoch 81/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0022 - val_loss: 0.0019
Epoch 82/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0019 - val_loss: 0.0020
Epoch 83/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0020 - val_loss: 0.0019
Epoch 84/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0020 - val_loss: 0.0021
Epoch 85/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 86/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0019 - val_loss: 0.0017
Epoch 87/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0019 - val_loss: 0.0023
Epoch 88/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 89/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 90/100
123/123 [==============================] - 0s 4ms/step - loss: 0.0019

(3911, 5, 3, 3)

In [4]:
accident_state = './cap_data/accident2.csv'
data2 = pd.read_csv(accident_state, sep=',', header=None)
accident1 = CsvToTensor(data2, tr_m_data, train=False)[0][0].reshape(1,5,3,3)
tr1 = X_train[0].reshape(1,5,3,3)
print(MSE_loss(tr1,AE_model))


tf.Tensor(0.00078685366, shape=(), dtype=float32)


In [6]:
accident_state = './cap_data/accident2.csv'
data2 = pd.read_csv(accident_state, sep=',', header=None)
accident1 = CsvToTensor(data2, tr_m_data, train=False)[0][0].reshape(1,5,3,3)
tr1 = X_train[0].reshape(1,5,3,3)
print(MSE_loss(accident1,AE_model))

tf.Tensor(3.651254, shape=(), dtype=float32)
